In [18]:
%load_ext autoreload
%autoreload 2

import sys
from pprint import pprint
import pandas as pd
import datetime
import time
import numpy as np

sys.path.append('../rest_api')
from context import rest_api
import rest_api.report_stats
import rest_api.station_stats

from pymongo import MongoClient
client = MongoClient('localhost:27017')
db = client.ContactDB

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from station_stats import  working_seconds_between_datetimes

In [20]:
station_id = "0276353b-58b8-4ad6-b4ec-1b6ab3f7741b"
product_group_id = "61ae37ba7e08b3abf7200fb4"

def aggregate_quantity(query, limit=None):
    pipeline = [{'$match': query}]
    pipeline.append({'$sort': {'move_datetime': -1}})
    if limit is not None:
        pipeline.append({'$limit': limit})
    pipeline.append({'$group': {
            '_id': '',
            'quantity': { '$sum': '$quantity' }
        }
    })
    pipeline.append({'$project': {
            '_id': 0,
            'totalQuantity': '$quantity'
        }
    })
    return db.touch_events.aggregate(pipeline)[0]['totalQuantity']

def calculate_pg_station_cycle_time(station_id, product_group_id, new_event):
    base_query = {'$and': [{'load_station_id':station_id}, {'product_group_id': product_group_id}]}

    recent_events = list(db.touch_events.find(base_query).sort('move_datetime', -1).limit(30))
    recent_events.append(new_event)
    first_event = list(db.touch_events.find(base_query).sort('move_datetime', 1).limit(1))
    
    if len(recent_events) > 2:
        first_event = first_event[0]
        
        recent_quantity = aggregate_quantity(base_query, limit=30) + new_event['quantity']
        total_quantity = aggregate_quantity(base_query, limit=None) + new_event['quantity']
        
        shift_details = db.settings.find_one()['shiftDetails']
        current_datetime = recent_events[0]['move_datetime']
        thirtieth_event_from_last_datetime = recent_events[-1]['move_datetime']
        first_datetime = first_event['move_datetime']
        
        total_working_seconds = working_seconds_between_datetimes(shift_details, first_datetime, current_datetime)
        recent_working_seconds = working_seconds_between_datetimes(shift_details, thirtieth_event_from_last_datetime, current_datetime)
        
        ps_historical_cycle_time = total_working_seconds / total_quantity
        ps_cycle_time = recent_working_seconds / recent_quantity
        
    else:
        return 0

calculate_pg_station_cycle_time(station_id, product_group_id)

TypeError: calculate_pg_station_cycle_time() missing 1 required positional argument: 'new_event'

In [12]:
base_query = {'$and': [{'load_station_id':station_id}, {'product_group_id': product_group_id}]}
list(db.touch_events.aggregate([
    {'$match': base_query},
    {'$sort': {'move_datetime': 1}},
    {'$limit': 30},
    {'$group': {
            '_id': '',
            'quantity': { '$sum': '$quantity' }
        }
    },
    {'$project': {
            '_id': 0,
            'totalQuantity': '$quantity'
        }
    }
]))[0]['totalQuantity']

278

In [17]:
list([e['move_datetime'] for e in db.touch_events.find().sort('move_datetime',1).limit(5)])

[datetime.datetime(2021, 8, 23, 21, 56, 17),
 datetime.datetime(2021, 8, 23, 21, 56, 17),
 datetime.datetime(2021, 8, 23, 21, 58, 50),
 datetime.datetime(2021, 8, 23, 21, 58, 50),
 datetime.datetime(2021, 8, 23, 22, 1, 23)]